In [131]:
import os
import pandas as pd


import eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
import eppy.json_functions as json_functions

# take in pandas of generation
# use eppy to load and edit base_idf file to fit children
# use run_many_EP from EP-wrapper to run whole generation and save to subfolders
# use eppy to read html results - https://eppy.readthedocs.io/en/latest/Outputs_Tutorial.html
# return html single kwh/m2 converted from MJ/m2 to function

In [132]:
iddfile = r"C:\EnergyPlusV22-2-0\Energy+.idd"
fname1 = r"W:\Insync\GDrive\Main\TU Delft\Thesis\EnergyPlus\Calibration files\Base_model.idf"
IDF.setiddname(iddfile)
idf1 = IDF(fname1)



In [133]:
idf1.printidf()


Version,
    22.2;                     !- Version Identifier

SimulationControl,
    Yes,                      !- Do Zone Sizing Calculation
    No,                       !- Do System Sizing Calculation
    Yes,                      !- Do Plant Sizing Calculation
    No,                       !- Run Simulation for Sizing Periods
    Yes,                      !- Run Simulation for Weather File Run Periods
    No,                       !- Do HVAC Sizing Simulation for Sizing Periods
    ;                         !- Maximum Number of HVAC Sizing Simulation Passes

Building,
    Building 1,               !- Name
    0,                        !- North Axis
    Suburbs,                  !- Terrain
    ,                         !- Loads Convergence Tolerance Value
    ,                         !- Temperature Convergence Tolerance Value
    FullExteriorWithReflections,    !- Solar Distribution
    ,                         !- Maximum Number of Warmup Days
    ;                         !- Mini

In [134]:

json_str = {"idf.VERSION..Version_Identifier":8.5,
            "idf.SIMULATIONCONTROL..Do_Zone_Sizing_Calculation": "No",
            "idf.SIMULATIONCONTROL..Do_System_Sizing_Calculation": "No",
            "idf.SIMULATIONCONTROL..Do_Plant_Sizing_Calculation": "No",
            "idf.BUILDING.Empire State Building.North_Axis": 52,
            "idf.BUILDING.Empire State Building.Terrain": "Rural",
            }
# json_functions.updateidf(idf1, json_str)

In [135]:
print(json_str)
df_whole = pd.read_csv(r"C:\Users\sebas\Documents\GitHub\ClimAIte\data_records\record_9", sep=',')

{'idf.VERSION..Version_Identifier': 8.5, 'idf.SIMULATIONCONTROL..Do_Zone_Sizing_Calculation': 'No', 'idf.SIMULATIONCONTROL..Do_System_Sizing_Calculation': 'No', 'idf.SIMULATIONCONTROL..Do_Plant_Sizing_Calculation': 'No', 'idf.BUILDING.Empire State Building.North_Axis': 52, 'idf.BUILDING.Empire State Building.Terrain': 'Rural'}


In [136]:
df_whole['idf.ZoneHVAC:IdealLoadsAirSystem.Z2_First_Floor_9e34b4d3 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness'] = df_whole['idf.ZoneHVAC:IdealLoadsAirSystem.Z1_Ground_Floor_48970ba6 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness']
df = df_whole.filter(regex='idf', axis=1)
df.head()
list_of_strings = []
keys = df.columns.values.tolist()
index_start = df.index.values.tolist()[0]
for row in range(df.shape[0]):
        dict = {}
        for i in keys:
                dict[i] = df.at[index_start + row, i]
        list_of_strings.append(dict)
print(list_of_strings[1])
print(index_start)

{'idf.People.People_5a9ee43c.People_per_Floor_Area': 0.12, 'idf.ElectricEquipment.Equipment_d01c6929.Watts_per_Zone_Floor_Area': 5, 'idf.ZoneInfiltration:DesignFlowRate.Infiltration_d1fbaf75.Flow_Rate_per_Exterior_Surface_Area': 0.00012, 'idf.WindowMaterial:SimpleGlazingSystem.PH_Window.UFactor': 1.0, 'idf.WindowMaterial:SimpleGlazingSystem.PH_Window.Solar_Heat_Gain_Coefficient': 0.9, 'idf.Material:NoMass.Typical Insulation-R50.Thermal_Resistance': 20, 'idf.Material.Generic Gypsum Board.Specific_Heat': 1300, 'idf.ZoneHVAC:IdealLoadsAirSystem.Z1_Ground_Floor_48970ba6 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness': 0.8, 'idf.ZoneHVAC:IdealLoadsAirSystem.Z2_First_Floor_9e34b4d3 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness': 0.8}
0


In [137]:
df.head()

,idf.People.People_5a9ee43c.People_per_Floor_Area,idf.ElectricEquipment.Equipment_d01c6929.Watts_per_Zone_Floor_Area,idf.ZoneInfiltration:DesignFlowRate.Infiltration_d1fbaf75.Flow_Rate_per_Exterior_Surface_Area,idf.WindowMaterial:SimpleGlazingSystem.PH_Window.UFactor,idf.WindowMaterial:SimpleGlazingSystem.PH_Window.Solar_Heat_Gain_Coefficient,idf.Material:NoMass.Typical Insulation-R50.Thermal_Resistance,idf.Material.Generic Gypsum Board.Specific_Heat,idf.ZoneHVAC:IdealLoadsAirSystem.Z1_Ground_Floor_48970ba6 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness,idf.ZoneHVAC:IdealLoadsAirSystem.Z2_First_Floor_9e34b4d3 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness
0,0.10,11,0.00008,0.8,0.7,16,1900,0.95,0.95
1,0.12,5,0.00012,1.0,0.9,20,1300,0.80,0.80
2,0.07,7,0.00002,1.4,0.3,7,2300,0.95,0.95
3,0.06,4,0.00008,0.8,0.8,5,900,0.80,0.80
4,0.09,4,0.00028,0.4,0.5,8,1000,0.95,0.95


In [138]:
df.shape

(200, 9)

In [139]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(list_of_strings)

[   {   'idf.ElectricEquipment.Equipment_d01c6929.Watts_per_Zone_Floor_Area': 11,
        'idf.Material.Generic Gypsum Board.Specific_Heat': 1900,
        'idf.Material:NoMass.Typical Insulation-R50.Thermal_Resistance': 16,
        'idf.People.People_5a9ee43c.People_per_Floor_Area': 0.1,
        'idf.WindowMaterial:SimpleGlazingSystem.PH_Window.Solar_Heat_Gain_Coefficient': 0.7,
        'idf.WindowMaterial:SimpleGlazingSystem.PH_Window.UFactor': 0.8,
        'idf.ZoneHVAC:IdealLoadsAirSystem.Z1_Ground_Floor_48970ba6 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness': 0.95,
        'idf.ZoneHVAC:IdealLoadsAirSystem.Z2_First_Floor_9e34b4d3 Ideal Loads Air System.Sensible_Heat_Recovery_Effectiveness': 0.95,
        'idf.ZoneInfiltration:DesignFlowRate.Infiltration_d1fbaf75.Flow_Rate_per_Exterior_Surface_Area': 8e-05},
    {   'idf.ElectricEquipment.Equipment_d01c6929.Watts_per_Zone_Floor_Area': 5,
        'idf.Material.Generic Gypsum Board.Specific_Heat': 1300,
        'idf.Mate

In [140]:
# for i in df.shape[0]:
for i in range(3):
    current_name = df_whole.at[index_start + i, 'Unique Name']
    json_str = list_of_strings[i]
    json_functions.updateidf(idf1, json_str)
    idf1.saveas(f'W:\Insync\GDrive\Main\TU Delft\Thesis\EnergyPlus\Calibration files\IDFs\Gene_{current_name}.idf')